In [ ]:
import numpy as np
from config import HYPERPARAMS, ECONPARAMS
from training_seq import training
from calc_nash_monopoly import actions_dict
from testing_seq import testing
params = HYPERPARAMS['full_obs_NB']
eparams = ECONPARAMS['base_case']
# Economic parameters
MEAN_C = 1
MEAN_Q = 2
A0 = 1
MU = 1/2
nA = params['nA']
grid = nA # Higher values gives better approximation of nash/monopoly-profits
# TRAINING SETUP ######################################################
FIRMLIST = []
diff_range = np.linspace(MEAN_Q*0.9,MEAN_Q*1.1,5)

for c in [MEAN_C]:
    for q in [MEAN_Q]:
        q = round(q, 2)
        FIRMLIST.append({'cost': c, 'quality': q})

NASH_ACTIONS = actions_dict(nA, A0, MU, FIRMLIST, FIRMLIST, "nash")
MONOPOLY_ACTIONS = actions_dict(nA, A0, MU, FIRMLIST, FIRMLIST, "monopoly")

eparams = {
        'firmlist': FIRMLIST,
        'a0': A0,
        'mu': MU,
        'nash_actions': NASH_ACTIONS,
        'monopoly_actions': MONOPOLY_ACTIONS,
        'randomness': 0
        }

In [ ]:
### TESTING SETUP #####################################################
FIRMLIST_expanded = []
diff_range = np.linspace(MEAN_Q*0.9,MEAN_Q*1.1,10)

for c in [MEAN_C]:
    for q in diff_range:
        q = round(q, 2)
        FIRMLIST_expanded.append({'cost': c, 'quality': q})

NASH_ACTIONS = actions_dict(nA, A0, MU, FIRMLIST_expanded, FIRMLIST_expanded, "nash")
MONOPOLY_ACTIONS = actions_dict(nA, A0, MU, FIRMLIST_expanded, FIRMLIST_expanded, "monopoly")

eparams_exp = {
        'firmlist': FIRMLIST_expanded,
        'a0': A0,
        'mu': MU,
        'nash_actions': NASH_ACTIONS,
        'monopoly_actions': MONOPOLY_ACTIONS,
        'randomness': 0
        }

In [ ]:
##### Takes a long time to run #######################################
agent = training(params, eparams, 8)
testing(FIRMLIST, agent, params, eparams,'seq.csv', False)
testing(FIRMLIST_expanded, agent, params, eparams_exp,'seq_exp.csv', False)
######################################################################

In [1]:
import torch
from torch import optim
from dqn_model import Net
import numpy as np
from agent import Agent1 as Agent
from experience_buffer import ExperienceBuffer
from config import HYPERPARAMS
from calc_nash_monopoly import actions_dict
params = HYPERPARAMS['full_obs_NB']
from cont_bertrand import ContBertrand


REPLAY_SIZE = params['replay_size']
nA = params['nA']
LEARNING_RATE = params['learning_rate']
#LEARNING_RATE = 0.001
#nA = 7
MEAN_C = 1
MEAN_Q = 2
A0 = 1
MU = 1/2
FIRMLIST = []

for c in [MEAN_C]:
    for q in [MEAN_Q]:
        q = round(q, 2)
        FIRMLIST.append({'cost': c, 'quality': q})
NASH_ACTIONS = actions_dict(nA, A0, MU, FIRMLIST, FIRMLIST, "nash")
MONOPOLY_ACTIONS = actions_dict(nA, A0, MU, FIRMLIST, FIRMLIST, "monopoly")


eparams = {
        'firmlist': FIRMLIST,
        'a0': A0,
        'mu': MU,
        'nash_actions': NASH_ACTIONS,
        'monopoly_actions': MONOPOLY_ACTIONS,
        'randomness': 0
        }

firm0 = FIRMLIST[0]
firm1 = firm0
PATH = "3499993_0checkpoint.pt"
checkpoint = torch.load(PATH)
model = Net(2,nA,8).to('cpu')
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE) 
model.load_state_dict(checkpoint['agent_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_dict'])
buffer = ExperienceBuffer(REPLAY_SIZE)
env = ContBertrand(firm0, firm1, eparams)
model.eval()
agent = Agent(env, buffer, model, model, optimizer)

In [3]:
from testing_seq import testing
testing(FIRMLIST, agent, params, eparams,'cheat.csv', True)